#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [60]:
# Alexa Aguilar Izquierdo
# Antonio Oldair Jimenez

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

    0) The binary result of quality assessment. 0=bad quality 1=sufficient quality.

    1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

    2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

    8-15) Contains the same information as 2-7, but for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the number of lesions with the diameter of the region of interest (ROI) to compensate for different image sizes. 

    16) The euclidean distance between the center of the macula and the center of the optic disc. This feature is also normalized with the diameter of the ROI.

    17) The diameter of the optic disc. 

    18) Result of the AM/FM-based (amplitude-modulation frequency-modulation) imaging. 0=normal and 1=abnormal.

    19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change these. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [61]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
from math import log2
import time
from numpy import mean

In [62]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q1. Implement the function `make_prediction` that takes the decision tree root and a data point instance and returns the prediction label.

In [63]:
def make_prediction(tree_root, data_point):
    if tree_root.is_leaf:
        prediction = tree_root.prediction
        return tree_root.prediction
    else:
        feature = tree_root.feature_idx
        if data_point.iloc[feature] < tree_root.thresh_val:
            return make_prediction(tree_root.left_child, data_point)
        else:
            return make_prediction(tree_root.right_child, data_point)

Q2. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [64]:
def split_dataset(data, feature_idx, threshold):
    left_split = data[data.iloc[:,feature_idx] < threshold]
    right_split = data[data.iloc[:,feature_idx] >= threshold]
    return (left_split, right_split)

Q3. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [65]:
def calc_entropy(data) -> float:
    entropy = 0.0
    #your code goes here
    left, right = split_dataset(data, -1, 0.5)

    if len(left) == 0 or len(right) == 0:
        return entropy

    def sub_entropy(sub):
        return -1 * (len(sub) / len(data)) * (log2((len(sub)) / len(data)))
        
    entropy = sub_entropy(left) + sub_entropy(right)
    return entropy

Q4. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [66]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
    #your code goes here
 
    data = data.sort_values(by=feature_idx)
    data = data.reset_index(drop=True)
 
    previous = None
 
    def info_gain(threshold):
        # entropy parent - sum of entropy child nodes
        left, right = split_dataset(data, feature_idx, threshold)
        entropy_left = calc_entropy(left) * (len(left)/len(data))
        entropy_right = calc_entropy(right) * (len(right)/len(data))
        return calc_entropy(data) - entropy_left - entropy_right
   
    for i, value in enumerate(data.iloc[:, -1]):
        if i == 0: # skip checking threhold of 0 and -1
            previous = value
            continue
        if value == previous: # no threshold
            continue
        else: # threshold
            threshold = (data.iloc[i, feature_idx] + data.iloc[i-1, feature_idx]) / 2
            info_gain_threshold = info_gain(threshold)
            if info_gain_threshold > best_info_gain:
                best_info_gain = info_gain_threshold
                best_thresh = threshold    
        previous = value
    return (best_info_gain, best_thresh)

Q5. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [67]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
        
    best_feature = None
    best_thresh = None
    best_info_gain, best_thresh = calc_best_threshold(data, 0)
    #your code goes here
    for feature_idx in range(len(data.columns) - 1):
        if feature_idx == 0:
            best_feature = 0
        else:
            info_gain, thresh = calc_best_threshold(data, feature_idx)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_feature = feature_idx
                best_thresh = thresh
   
    return (best_feature, best_thresh)

Q6. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [68]:
def create_leaf_node(data):        
    #your code goes here
    # is_leaf = True          # boolean variable to check if the node is a leaf
    # feature_idx = None      # index that identifies the feature
    # thresh_val = None       # threshold value that splits the node
    # prediction = None       # prediction class (only valid for leaf nodes)
    # left_child = None       # left TreeNode (all values < thresh_val)
    # right_child = None      # right TreeNode (all values >= thresh_val)

    leaf_node = TreeNode()
    leaf_node.is_leaf = True
    no_disease = len(data[data.iloc[:, -1] == 0])    
    disease = len(data[data.iloc[:, -1] == 1])
    if no_disease > disease:
        leaf_node.prediction = 0
    else:
        leaf_node.prediction = 1
    return leaf_node

Q7. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree - For example, if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [69]:

def create_decision_tree(data, max_levels):

    def recursive_tree(data, i):
        if data is not None:
            best_feature, best_thresh = identify_best_split(data)
            
            if i == max_levels or best_thresh is None:
                leaf = create_leaf_node(data)
                return leaf
            else:
                left, right = split_dataset(data, best_feature, best_thresh)
                node = TreeNode()
                node.is_leaf = False
                node.feature_idx = best_feature
                node.thresh_val = best_thresh
                
                node.right_child = recursive_tree(right, i+1)
                node.left_child = recursive_tree(left, i+1)
                return node
        else:
            return 
 
    root = recursive_tree(data, 1)

    return root
    
    

Q8. Given a decision tree and a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [70]:
def calc_accuracy(tree_root, test_data):
    correct = 0
   
    for i in range(len(test_data)):
        prediction = make_prediction(tree_root, test_data.iloc[i])
        if prediction == test_data.iloc[i,-1]:
            correct += 1
           
    return correct/len(test_data)

Q9. Now measure the accuracy of using a decision tree on this data with a 5-fold cross validation. 
Set the `max_levels` parameter to 10 and print the accuracy from a 5-fold-CV.

This must run in under 10 minutes, otherwise points will be deducted. 

In [71]:
def run_CV(filename):
    start = time.time()
    
    # read in data
    d = pd.read_csv(filename, header = None)

    #your code goes here
    chunk_size = int(d.shape[0] / 5)
    accuracies = []
    for start in range(0, d.shape[0], chunk_size):
        d_validate = d.iloc[start:start + chunk_size] # extract chunk for testing
        # calculate accuracy
        d_temp = d.drop(d_validate.index)
        tree_root = create_decision_tree(d_temp, 10)
        tree_root.printTree()
        accuracies.append(calc_accuracy(tree_root, d_validate))

    five_fold_accruacy = mean(accuracies)
    print(five_fold_accruacy)
    end = time.time()
    print ('Time taken:', end - start)


run_CV("messidor_features.txt")

Internal Node:  splits on feature 14 with threshold 0.059243000000000004
-Internal Node:  splits on feature 2 with threshold 69.0
--Internal Node:  splits on feature 8 with threshold 147.875086
---Internal Node:  splits on feature 8 with threshold 30.0813355
----Internal Node:  splits on feature 17 with threshold 0.0866005
-----Leaf Node:      predicts 1
-----Internal Node:  splits on feature 2 with threshold 55.5
------Internal Node:  splits on feature 8 with threshold 29.553185499999998
-------Internal Node:  splits on feature 11 with threshold 0.10185649999999999
--------Internal Node:  splits on feature 8 with threshold 27.3275705
---------Leaf Node:      predicts 0
---------Leaf Node:      predicts 0
--------Internal Node:  splits on feature 8 with threshold 11.194698500000001
---------Leaf Node:      predicts 1
---------Leaf Node:      predicts 0
-------Leaf Node:      predicts 1
------Internal Node:  splits on feature 5 with threshold 57.0
-------Internal Node:  splits on featur